In [1]:
%pip install langchain==0.3.14
%pip install langchain-openai==0.2.14
%pip install youtube-transcript-api
%pip install pytube


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langchain_community.document_loaders import YoutubeLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import AzureChatOpenAI

In [7]:
import os

from dotenv import load_dotenv

load_dotenv(".env", override=True)

True

In [10]:
youtube_url= "https://www.youtube.com/watch?v=7-7FJcah1iA"

loader = YoutubeLoader.from_youtube_url(
    youtube_url,
    # add_video_info=True,
    language=["ja"])

# ドキュメントを読み込む
documents = loader.load()
print(documents[0].page_content)

花風を引いてますけど鼻あ花風を引いて ますけどあのも題なくえっと問題ない問題 ないというかうん元気で元気ですよ少し だけ疲れてますけどそうなんですなんかで も前回よりげししてないですねあそうあの あの僕も思いましたなんか前回少し ふっくらしたなと思いましたでも体重は別 に増えたわけじゃ なく健康的な感じじゃないですかお そうですね確かにあの70は超えてない です69.何ぼとかって時々あるんです けどはい70はいかないですね68と69 の間を行ったり来たりしてますそうです なんか1番減った時とそんなに今変わって ないのではいやっぱ食べてるか食べてない かでねやっぱゲストリドが変わるんだなっ てすいませんあ本当ですね面白いです ね思いましたこんなに変るって今見て思い ます結構変わりやすいのかもしれないです ねそうれでくと確かに顔に出やすい方とか いらっしゃいますよ ねうんそうです多分ま僕嘘つけないタイプ な分顔に出るだと思いますそっちそっちの ですね本体調とかも全部顔出てああでも あの分かりやすくてれはいいです確か に確か にたと最近のお食事のはいまだ遅れて なかったんですけど奥さんにあの分析の 結果を出したの でしたいんだ えっとこの12123って書いてあんです が13なのかは日日が書いてなかったので ちょっと分からない状態ではい3日間3 日間です12とその間にあった食事の中で 朝晩ピックアップしてあと23日っていう 感じ あのしましたわかりましたはいなのでまだ こうあれですねあの回復してすぐのところ のお食事なので野菜の量としてはいつもの 量ではないていう158になってるの でちょっとあれあそうなんですまだやっぱ 食べないとなんですね野菜は結構多くなっ たと思ったんですけどあそうですか1番 食べてる時に比べる1番食べてる時だと 多分200とか250とか行ってるかなと 思うけどあそうなんですねはいちょっと 控えてた時期も含めてって感じなのでああ 分かりましたなるほど今の倍食べて いただいても300ちょっとなのであの倍 食べても大丈夫 ですそんなに食べれ ない本当ですかなんか割とあのやり始める と400とか行く方とかもいらっしゃるん ですよどうやって増やすんですかねその 自分自炊をしないからあありがとうやどう しようなんかあれですかね結構野菜サラダ 高いじゃないですかあ

In [18]:
doc = documents[0].page_content

llm = AzureChatOpenAI(
            openai_api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            azure_deployment=os.getenv("ZURE_OPENAI_GPT4_DEPLOYMENT_NAME"),
            openai_api_version="2023-09-15-preview",
            temperature=0.5
        )

summary_prompt_template = """
以下のYouTube動画の書き起こしをユーザーと管理栄養士に分類した会話に整理してください:
{text}
"""

summary_prompt = PromptTemplate.from_template(
    summary_prompt_template)
# チェーン
summary = summary_prompt | llm | StrOutputParser()
result = summary.invoke(doc)
print(result)

NotFoundError: Error code: 404 - {'error': {'code': '404', 'message': 'Resource not found'}}

In [ ]:
llm = AzureChatOpenAI(
            openai_api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            azure_deployment=os.getenv("ZURE_OPENAI_GPT4_DEPLOYMENT_NAME"),
            openai_api_version="2023-09-15-preview",
            temperature=0.5
        )

evaluation_prompt_template = """
以下のユーザーと管理栄養士の会話を評価してください:
{text}
"""
    
evaluation_prompt = PromptTemplate.from_template(
    evaluation_prompt_template)

# チェーン
chain = evaluation_prompt | llm | StrOutputParser()
result = chain.invoke(content)